In [1]:
#script to run simulations of all the mechanisms made under the different rxn DFT uncertainty assumptions

In [2]:
import os
import cantera as ct
import numpy as np
import pandas as pd
import concurrent.futures


import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
yaml_path = 'global path'

In [6]:
# Load the experimental conditions
ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
df_exp = pd.read_csv(ignition_delay_data)

# slice just table 7, where phi=1.0
table7 = df_exp[df_exp['Table'] == 7]
# Define Initial conditions using experimental data
tau7 = table7['time (ms)'].values.astype(float)  # ignition delay
T7 = table7['T_C'].values  # Temperatures
P7 = table7['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm


# list of starting conditions
# Mixture compositions taken from table 2 of
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
concentrations = []
# for phi = 1
x_diluent = 0.7649
conc_dict = {
    'O2(2)': 0.2038,
    'butane(1)': 0.03135
}

for i in range(0, len(table7)):
    x_N2 = table7['%N2'].values[i] / 100.0 * x_diluent
    x_Ar = table7['%Ar'].values[i] / 100.0 * x_diluent
    x_CO2 = table7['%CO2'].values[i] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    conc_dict['CO2(7)'] = x_CO2
    concentrations.append(conc_dict)

    
# # Aramco naming:
# x_diluent = 0.7649
# conc_dict = {
#     'O2': 0.2038,
#     'C4H10': 0.03135
# }

# for i in range(0, len(table7)):
#     x_N2 = table7['%N2'].values[i] / 100.0 * x_diluent
#     x_Ar = table7['%Ar'].values[i] / 100.0 * x_diluent
#     x_CO2 = table7['%CO2'].values[i] / 100.0 * x_diluent
#     conc_dict['N2'] = x_N2
#     conc_dict['AR'] = x_Ar
#     conc_dict['CO2'] = x_CO2
#     concentrations.append(conc_dict)


In [11]:
# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def get_delay(gas, T, P, X):
    # function to run a RCM simulation

    t_end = 1.0  # time in seconds
    gas.TPX = T, P, X

    env = ct.Reservoir(ct.Solution('air.yaml'))
    # env = ct.Reservoir(ct.Solution('air.xml'))
    reactor = ct.IdealGasReactor(gas)
    wall = ct.Wall(reactor, env, A=1.0, velocity=0)
    reactor_net = ct.ReactorNet([reactor])
    # # allegedly faster solving
    # reactor_net.derivative_settings = {"skip-third-bodies": True, "skip-falloff": True}
    # reactor_net.preconditioner = ct.AdaptivePreconditioner()

    times = [0]
    T = [reactor.T]
    P = [reactor.thermo.P]
    X = [reactor.thermo.X]  # mol fractions
    while reactor_net.time < t_end:
        reactor_net.step()

        times.append(reactor_net.time)
        T.append(reactor.T)
        P.append(reactor.thermo.P)
        X.append(reactor.thermo.X)

    slopes = np.gradient(P, times)
    i = np.argmax(slopes)
    return times[i]

In [12]:
def run_simulation(condition_index):
    gas = ct.Solution(yaml_path)
    X = concentrations[condition_index]
    delay = get_delay(gas, T7[condition_index], P7[condition_index], X)
    print(f'Completed {condition_index}:\t {delay}')
    return delay

In [13]:
basedir = '/work/westgroup/harris.se/autoscience/reaction_calculator/rxn_dft_uncertainty_experiment/'

In [14]:
uncertainties = np.round(np.arange(0.0, 5.1, 0.1), 1)

In [15]:
for dft_uncertainty in uncertainties:
    output_path = os.path.join(basedir, f'ignition_delay_{dft_uncertainty}.npy')
    if os.path.exists(output_path):
        continue
    
    print(f'Running simulation for uncertainty={dft_uncertainty}')
    yaml_path = os.path.join(basedir, f'uncertainty_{dft_uncertainty}.yaml')
    gas = ct.Solution(yaml_path)
    
    
    # Run all simulations in parallel
    delays = np.zeros(len(table7))
    condition_indices = np.arange(0, len(table7))
    with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
        for condition_index, delay_time in zip(condition_indices, executor.map(run_simulation, condition_indices)):
            delays[condition_index] = delay_time
            
    # save the delays as an npy file
    np.save(output_path, delays)

Running simulation for uncertainty=0.0
Completed 15:	 0.015265614510390376
Completed 13:	 0.026942890787600062
Completed 14:	 0.02553176559413088
Completed 7:	 0.218144921697427
Completed 9:	 0.1109008971106346Completed 11:	 0.06203230888327197
Completed 10:	 0.07334130426966373

Completed 12:	 0.038864526058477816
Completed 8:	 0.12900514131072838
Completed 6:	 0.219019592782658
Completed 4:	 0.1654147745077036
Completed 5:	 0.16815025952779158
Completed 2:	 0.10767659179999393
Completed 3:	 0.10934373620982285
Completed 0:	 0.08726184865851735
Completed 1:	 0.08699176307344607
Running simulation for uncertainty=0.1
Completed 15:	 0.015265614510390376
Completed 13:	 0.026942890787600062
Completed 14:	 0.02553176559413088
Completed 11:	 0.06203230888327197
Completed 9:	 0.1109008971106346Completed 7:	 0.218144921697427

Completed 10:	 0.07334130426966373
Completed 12:	 0.038864526058477816
Completed 8:	 0.12900514131072838
Completed 6:	 0.219019592782658
Completed 4:	 0.165414774507703

KeyboardInterrupt: 